#**Обучаем и сохраняем все ML-модели; пробуем классифицировать произвольный текст**

**Устанавливаем и подключаем библиотеки**

In [ ]:
!pip -q install pymorphy2
!pip -q install sentencepiece

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 61.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np


In [ ]:
from sklearn.model_selection import train_test_split # делим на тестовую и тренировочную
from sklearn.feature_extraction.text import CountVectorizer # мешок слов
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report # метрика точности, f1 мера, матрица ошибок и отчёт по классификации
from sklearn import model_selection # выбор модели и оценка параметров (можно загрузить несколько моделей)
from sklearn.linear_model import LogisticRegression # логистическая регрессия
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier # бустинг, случайный лес
from sklearn.svm import SVC # метод SVM, загружаем с ядром rbf https://scikit-learn.ru/1-4-support-vector-machines/
from sklearn.neighbors import KNeighborsClassifier # к ближайших соседей

In [ ]:
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import pymorphy2
import re

In [ ]:
# Загрузка стоп-слов и инициализация лемматизатора
nltk.download('stopwords')
russian_stopwords = set(stopwords.words('russian'))
morph = pymorphy2.MorphAnalyzer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Функция для предобработки текста
def preprocess_text(text):
    # Приведение к нижнему регистру и удаление пунктуации
    text = re.sub(r'[^\w\s]', '', text.lower())
    # Токенизация и лемматизация
    tokens = text.split()
    lemmatized = [morph.parse(token)[0].normal_form for token in tokens if token not in russian_stopwords]
    return ' '.join(lemmatized)

**Загружаем датасет с очищенным текстом**

In [ ]:
!gdown 11KFmtEdK_h1S5RyfRfHSp4fJBDEyo0NI

Downloading...
From: https://drive.google.com/uc?id=11KFmtEdK_h1S5RyfRfHSp4fJBDEyo0NI
To: /content/clean_df.csv
100% 6.87M/6.87M [00:00<00:00, 134MB/s]


In [ ]:
df = pd.read_csv('clean_df.csv')

In [ ]:
df

,text,label
0,введение цель выпускной квалификационный работ...,0
1,работа представить ниже стоить считать предста...,0
2,компьютерный игра правило игровой ситуация вос...,0
3,сразу стоить отметить сегодняшний день существ...,0
4,вовторое разработчик прекращать свой экспериме...,0
...,...,...
5920,воздействие внешний условие превращаться орган...,1
5921,современный настольный пк такой компонент прак...,1
5922,стоимость такой услуга большой это безусловно ...,1
5923,использовать пример разработка аналогичный сис...,1


**Делим выборку на train и test**

In [ ]:
test_size = 0.3

X_train, X_test, y_train, y_test = train_test_split(df['text'],
                                                    df['label'],
                                                    test_size=test_size,
                                                    random_state=42)


**Функции для сохранения и загрузки моделей и векторизатора**

In [ ]:
def get_models():
    return {
        'RandomForest': RandomForestClassifier(bootstrap=True, n_estimators=100, n_jobs=-1),
        "GB": GradientBoostingClassifier(n_estimators=100),
        "kNN": KNeighborsClassifier(),
        "LogisticRegression": LogisticRegression(),
        "SVC": SVC(kernel='rbf')
    }

# Функция для сохранения моделей и векторизатора
def save_models(models, vectorizer):
    for name, model in models.items():
        with open(f'{name}_model.pkl', 'wb') as f:
            pickle.dump(model, f)

    with open('vectorizer.pkl', 'wb') as f:
        pickle.dump(vectorizer, f)

# Функция для загрузки моделей и векторизатора
def load_models():
    loaded_models = {}
    model_names = ['RandomForest', 'GB', 'kNN', 'LogisticRegression', 'SVC']

    for name in model_names:
        with open(f'{name}_model.pkl', 'rb') as f:
            loaded_models[name] = pickle.load(f)

    with open('vectorizer.pkl', 'rb') as f:
        vectorizer = pickle.load(f)

    return loaded_models, vectorizer



**Векторизуем текст с помощью TF-IDF векторизатора**

In [ ]:
# Основной код
vectorizer = TfidfVectorizer()

X_train_TFIDF = vectorizer.fit_transform(X_train)
X_test_TFIDF = vectorizer.transform(X_test)

**Обучаем и сохраняем модели**

In [ ]:
metrics = {}
models = get_models()
for name, model in models.items():
    print(name)
    clf = model.fit(X_train_TFIDF, y_train)
    y_predict_TFIDF = clf.predict(X_test_TFIDF)
    score = f1_score(y_predict_TFIDF, y_test)
    metrics[name] = score

# Сохранение обученных моделей и векторизатора
save_models(models, vectorizer)


RandomForest
GB
kNN
LogisticRegression
SVC


**Оцениваем качество моделей**

In [ ]:
df_metrics = pd.DataFrame.from_dict(metrics, orient='index').reset_index()
df_metrics.columns = ['model', 'f1_score']

In [ ]:
df_metrics

,model,f1_score
0,RandomForest,0.801853
1,GB,0.759892
2,kNN,0.825248
3,LogisticRegression,0.836903
4,SVC,0.852696


**Загружаем модели и классифицируем произвольный текст**

In [ ]:
# Пример использования сохраненных моделей
loaded_models, loaded_vectorizer = load_models()

In [ ]:
paragraph = ["Пользователи, или клиенты, могут отправлять, получать, просматривать и управлять документами через специальные приложения, которые взаимодействуют с сервером ис учёта ресурсов. Таким образом, ис учёта ресурсов обычно использует клиентсерверную архитектуру, где сервер обрабатывает запросы на поставку и хранение материалов, а клиенты работают с этой системой через свои приложения. Клиентсерверная архитектура в ис учёта ресурсов предприятия обеспечивает удобство и эффективность для пользователей. Клиенты могут использовать специализированные приложения для взаимодействия с сервером, отправлять. Сервер, в свою очередь, обеспечивает централизованное хранение данных, обработку запросов, контроль доступа и безопасность информации."]

In [ ]:
preprocessed_paragraph = preprocess_text(paragraph)

In [ ]:
# Вывод результата для проверки
print(preprocessed_paragraph)

пользователь клиент мочь отправлять получать просматривать управлять документ специальный приложение который взаимодействовать сервер иса учёт ресурс такой образ иса учёт ресурс обычно использовать клиентсерверный архитектура сервер обрабатывать запрос поставка хранение материал клиент работать система свой приложение клиентсерверный архитектура иса учёт ресурс предприятие обеспечивать удобство эффективность пользователь клиент мочь использовать специализировать приложение взаимодействие сервер отправлять сервер очередь обеспечивать централизовать хранение данные обработка запрос контроль доступ безопасность информация


In [ ]:
# Предсказание на новых данных
new_data = [preprocessed_paragraph]
new_data_vectorized = loaded_vectorizer.transform(new_data)

**результаты классификации произвольного текста для разных моделей на признаках TF-IDF**

In [ ]:
for name, model in loaded_models.items():
    prediction = model.predict(new_data_vectorized)
    print(f"Предсказание модели {name}: {prediction}")

Предсказание модели RandomForest: [1]
Предсказание модели GB: [1]
Предсказание модели kNN: [1]
Предсказание модели LogisticRegression: [1]
Предсказание модели SVC: [1]


===================================================================================================================

#Классификация текста на новых числовых признаках

**Установка библиотек**

In [ ]:
!pip install spacy

In [ ]:
!python -m spacy download ru_core_news_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 30.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import pandas as pd
import numpy as np
import nltk
import spacy
import math
from collections import Counter
from nltk.util import ngrams
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import re

In [ ]:
import sklearn

In [ ]:
from sklearn.model_selection import train_test_split # делим на тестовую и тренировочную
from sklearn.feature_extraction.text import CountVectorizer # мешок слов


from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report # метрика точности, f1 мера, матрица ошибок и отчёт по классификации
from sklearn import model_selection # выбор модели и оценка параметров (можно загрузить несколько моделей)
from sklearn.linear_model import LogisticRegression # логистическая регрессия
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier # бустинг, случайный лес
from sklearn.svm import SVC # метод SVM, загружаем с ядром rbf https://scikit-learn.ru/1-4-support-vector-machines/
from sklearn.neighbors import KNeighborsClassifier # к ближайших соседей

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [ ]:
# Загрузка необходимых ресурсов
nltk.download('punkt')
nltk.download('stopwords')
nlp = spacy.load('ru_core_news_sm')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**Функции для расчета новых числовых признаков (нужны для классификации произвольного текста)**

In [ ]:
# Функция для подсчета энтропии распределения символов
def calculate_entropy(text):
    freq = Counter(text)
    entropy = 0
    for char, count in freq.items():
        prob = count / len(text)
        entropy -= prob * math.log2(prob)
    return entropy

# Функция для подсчета энтропии распределения слов
def calculate_word_entropy(tokens):
    freq = Counter(tokens)
    total_words = len(tokens)
    entropy = 0
    for word, count in freq.items():
        prob = count / total_words
        entropy -= prob * math.log2(prob)
    return entropy


# Функция для подсчета TTR
def calculate_ttr(tokens):
    return len(set(tokens)) / len(tokens)

# Функция для подсчета индекса Хердана
def calculate_herdan_index(tokens):
    return math.log(len(set(tokens))) / math.log(len(tokens))


# Функция для подсчета индекса Юла
def calculate_yule_index(tokens):
    freq = Counter(tokens)
    m1 = sum(freq.values())
    freq_of_freq = Counter(freq.values())
    m2 = sum([f * (f - 1) * freq_of_freq[f] for f in freq_of_freq])
    return 10000 * (m2) / (m1 * m1 - m1)


# Функция для подсчета частоты использования служебных слов
def calculate_function_words_ratio(tokens):
    stop_words = set(stopwords.words('russian'))
    return sum(1 for token in tokens if token.lower() in stop_words) / len(tokens)

# Функция для подсчета средней глубины синтаксического дерева
def calculate_avg_tree_depth(doc):
    def get_depth(token):
        depth = 0
        while token.head != token:
            token = token.head
            depth += 1
        return depth

    sentence_depths = []
    for sent in doc.sents:
        depths = [get_depth(token) for token in sent if token.dep_ != 'ROOT']
        if depths:
            sentence_depths.append(sum(depths) / len(depths))

    return sum(sentence_depths) / len(sentence_depths) if sentence_depths else 0


# Функция для подсчета соотношения частей речи
def calculate_pos_ratio(doc):
    pos_counts = Counter(token.pos_ for token in doc)
    total = sum(pos_counts.values())
    if total == 0:
        return {}  # Возвращаем пустой словарь, если текст пустой

    # Создаем словарь со всеми возможными частями речи, инициализируя их нулями
    all_pos = {pos: 0 for pos in spacy.parts_of_speech.IDS}

    # Обновляем значения для найденных частей речи
    for pos, count in pos_counts.items():
        all_pos[pos] = count / total

    return all_pos


#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# Функция для подсчета частоты использования падежей - почему-то не работает (!!!)
def calculate_case_frequency(doc):
    case_counts = Counter(token.morph.get('Case') for token in doc if 'Case' in token.morph)
    total = sum(case_counts.values())
    return {case: count / total for case, count in case_counts.items()}

#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!1
# Функция для подсчета частоты биграмм и триграмм - исключила - дает слишком много малоинформативных признаков (!!!)
def calculate_ngram_frequency(tokens, n, top_n=2):
    ngram_counts = Counter(ngrams(tokens, n))
    total = sum(ngram_counts.values())
    return {f'top_{i+1}_{n}gram': ' '.join(ngram) for i, (ngram, _) in enumerate(ngram_counts.most_common(top_n))}


# Функции для расчета индекса удобочитаемости Флеша-Кинкейда (адаптированная для русского языка)
#функция count_syllables_ru для подсчета слогов в русских словах
def count_syllables(word):
    vowels = 'аеёиоуыэюя'
    return len([char for char in word.lower() if char in vowels])

def calculate_flesch_kincaid_grade(text):
    # Очистка текста
    text = re.sub(r'[^\w\s.,!?]', '', text)

    sentences = sent_tokenize(text)
    words = [word.strip('.,!?:;()[]{}') for word in word_tokenize(text) if word.strip('.,!?:;()[]{}')]

    if not sentences or not words:
        return 0

    avg_sentence_length = len(words) / len(sentences)
    avg_syllables_per_word = sum(count_syllables(word) for word in words) / len(words)

    # Адаптированные коэффициенты для русского языка
    score = 220.755 - (1.1 * avg_sentence_length) - (65.0 * avg_syllables_per_word)

    # Ограничение минимального значения
    return max(0, score)


# Функция для расчета индекса туманности Ганнинга (адаптированная для русского языка)
def calculate_gunning_fog_index(text):
    if not text:
        return 0

    # Очистка текста
    text = re.sub(r'[^\w\s]', '', text.lower())

    sentences = sent_tokenize(text)
    words = word_tokenize(text)

    if len(sentences) == 0 or len(words) == 0:
        return 0

    complex_words = sum(1 for word in words if count_syllables(word) >= 4)

    return 0.4 * ((len(words) / len(sentences)) + 100 * (complex_words / len(words)))

# Функция для подсчета частоты повторения слов
def calculate_word_repetition(tokens):
    freq = Counter(tokens)
    return sum(count for count in freq.values() if count > 1) / len(tokens)

**Применяем функции к датафрейму (нужны при классификации произвольного текста)**

In [ ]:
# Применение функций к датафрейму
def extract_features(df):
    features = []
    for _, row in df.iterrows():
        text = row['content']
        tokens = word_tokenize(text.lower())
        sentences = sent_tokenize(text)
        doc = nlp(text)

        feature = {
            'char_count': len(text),
            'word_count': len(tokens),
            'sentence_count': len(sentences),
            'avg_word_length': sum(len(word) for word in tokens) / len(tokens),
            'avg_sentence_length': len(tokens) / len(sentences),
            'ttr': calculate_ttr(tokens),
            'herdan_index': calculate_herdan_index(tokens),
            'yule_index': calculate_yule_index(tokens),
            'function_words_ratio': calculate_function_words_ratio(tokens),
            'avg_punctuation_per_sentence': sum(1 for char in text if char in '.,;:!?') / len(sentences),
            'avg_tree_depth': calculate_avg_tree_depth(doc),
            'char_entropy': calculate_entropy(text),
            #'word_entropy': calculate_entropy(' '.join(tokens)),
            'word_entropy': calculate_word_entropy(tokens),
            'flesch_kincaid_grade': calculate_flesch_kincaid_grade(text),
            'gunning_fog_index': calculate_gunning_fog_index(text),
            'word_repetition_frequency': calculate_word_repetition(tokens),
        }

        pos_ratio = calculate_pos_ratio(doc)
        feature.update({f'pos_ratio_{pos}': ratio for pos, ratio in pos_ratio.items()})

        case_frequency = calculate_case_frequency(doc)
        feature.update({f'case_frequency_{case}': freq for case, freq in case_frequency.items()})

        #bigram_frequency = calculate_ngram_frequency(tokens, 2)
        #feature.update({f'bigram_frequency_{bigram}': freq for bigram, freq in bigram_frequency.items()})

        #trigram_frequency = calculate_ngram_frequency(tokens, 3)
        #feature.update({f'trigram_frequency_{trigram}': freq for trigram, freq in trigram_frequency.items()})

        features.append(feature)

    return pd.DataFrame(features)


**Загружаем готовый датасет с новыми числовыми признаками и меткой класса**

In [ ]:
!gdown 1CYtwcQYeKtQF1cAsfzQS3rT_P23-fW_j

Downloading...
From: https://drive.google.com/uc?id=1CYtwcQYeKtQF1cAsfzQS3rT_P23-fW_j
To: /content/gotov_df.csv
100% 2.68M/2.68M [00:00<00:00, 162MB/s]


In [ ]:
df = pd.read_csv('gotov_df.csv')

In [ ]:
X = df.drop('label', axis=1)
y = df['label']

**Делим на train и  test**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**Определяем пайплайн для моделей**

In [ ]:
def get_models1():
    return {
        'RandomForest1': RandomForestClassifier(bootstrap=True, n_estimators=100, n_jobs=-1),
        "GB1": GradientBoostingClassifier(n_estimators=100),
        "kNN1": Pipeline([
            ('scaler', StandardScaler()),
            ('knn', KNeighborsClassifier())
        ]),
        "LogisticRegression1": Pipeline([
            ('scaler', StandardScaler()),
            ('lr', LogisticRegression())
        ]),
        "SVC1": Pipeline([
            ('scaler', StandardScaler()),
            ('svc', SVC(kernel='rbf'))
        ])
    }

**Обучаем модели и оцениваем качество**

In [ ]:
metrics = {}
models = get_models1()

for name, model in models.items():
    print(f"Обучение модели: {name}")
    clf = model.fit(X_train, y_train)
    y_predict = clf.predict(X_test)
    score = f1_score(y_test, y_predict)
    metrics[name] = score
    print(f"F1-score для {name}: {score:.4f}")

Обучение модели: RandomForest1
F1-score для RandomForest1: 0.6968
Обучение модели: GB1
F1-score для GB1: 0.6999
Обучение модели: kNN1
F1-score для kNN1: 0.6804
Обучение модели: LogisticRegression1
F1-score для LogisticRegression1: 0.7030
Обучение модели: SVC1
F1-score для SVC1: 0.7157


**Догружаем необходимые ресурсы**

In [ ]:
import pickle
import joblib
from sklearn.base import BaseEstimator, TransformerMixin

**Здесь функции для извлечения признаков из произвольного текста, сохранения и загрузки моделей**

In [ ]:
# Класс для извлечения признаков
class FeatureExtractor(BaseEstimator, TransformerMixin):
    def __init__(self):
        # Инициализация необходимых ресурсов
        nltk.download('punkt')
        nltk.download('stopwords')
        self.nlp = spacy.load('ru_core_news_sm')

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return extract_features(pd.DataFrame({'content': X}))

# Сохранение моделей
def save_models1(models):
    for name, model in models.items():
        filename = f"{name}_model1.pkl"
        with open(filename, 'wb') as file:
            pickle.dump(model, file)
    print("Модели сохранены.")

# Загрузка моделей
def load_models1():
    loaded_models = {}
    for name in ['RandomForest1', 'GB1', 'kNN1', 'LogisticRegression1', 'SVC1']:
        filename = f"{name}_model1.pkl"
        with open(filename, 'rb') as file:
            loaded_models[name] = pickle.load(file)
    print("Модели загружены.")
    return loaded_models

# Функция для классификации произвольного абзаца
def classify_paragraph(paragraph, models):
    # Создаем DataFrame с одним абзацем
    df = pd.DataFrame({'content': [paragraph]})

    # Извлекаем признаки
    features = extract_features(df)

    # Выбираем нужные столбцы
    selected_features = [
        'char_count', 'word_count', 'sentence_count', 'avg_word_length',
        'avg_sentence_length', 'ttr', 'herdan_index', 'yule_index',
        'function_words_ratio', 'avg_punctuation_per_sentence', 'avg_tree_depth',
        'char_entropy', 'word_entropy', 'flesch_kincaid_grade', 'gunning_fog_index',
        'word_repetition_frequency', 'pos_ratio_ADJ', 'pos_ratio_ADP', 'pos_ratio_ADV',
        'pos_ratio_AUX', 'pos_ratio_CCONJ', 'pos_ratio_DET', 'pos_ratio_INTJ',
        'pos_ratio_NOUN', 'pos_ratio_NUM', 'pos_ratio_PART', 'pos_ratio_PRON',
        'pos_ratio_PROPN', 'pos_ratio_PUNCT', 'pos_ratio_SCONJ', 'pos_ratio_SYM',
        'pos_ratio_VERB', 'pos_ratio_X', 'pos_ratio_SPACE'
    ]

    features = features[selected_features]
    #print(features)
    # Классифицируем с помощью каждой модели
    results = {}
    for name, model in models.items():
        prediction = model.predict(features)
        results[name] = prediction[0]

    return results


**Сохраняем обученные модели**

In [ ]:
# Сохраняем обученные модели
save_models1(models)


Модели сохранены.


**Загружаем сохраненные модели**

In [ ]:
# Пример использования
loaded_models = load_models1()

Модели загружены.


**Классифицируем произвольный абзац**

In [ ]:
# Классифицируем произвольный абзац
sample_paragraph = "Пользователи, или клиенты, могут отправлять, получать, просматривать и управлять документами через специальные приложения, которые взаимодействуют с сервером ис учёта ресурсов. Таким образом, ис учёта ресурсов обычно использует клиентсерверную архитектуру, где сервер обрабатывает запросы на поставку и хранение материалов, а клиенты работают с этой системой через свои приложения. Клиентсерверная архитектура в ис учёта ресурсов предприятия обеспечивает удобство и эффективность для пользователей. Клиенты могут использовать специализированные приложения для взаимодействия с сервером, отправлять. Сервер, в свою очередь, обеспечивает централизованное хранение данных, обработку запросов, контроль доступа и безопасность информации."
classification_results = classify_paragraph(sample_paragraph, loaded_models)

In [ ]:
print("Результаты классификации:")
for model_name, prediction in classification_results.items():
    #print(f"{model_name}: {'Позитивный' if prediction == 1 else 'Негативный'}")
    print(f"{model_name}: {prediction}")

Результаты классификации:
RandomForest1: 1
GB1: 1
kNN1: 0
LogisticRegression1: 1
SVC1: 1
